In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

--2024-01-18 07:49:57--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  17.8MB/s    in 3.4s    

2024-01-18 07:50:01 (17.8 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
%cd "/content/"
!git clone https://github.com/mpc001/Visual_Speech_Recognition_for_Multiple_Languages.git
%cd "Visual_Speech_Recognition_for_Multiple_Languages"

/content
Cloning into 'Visual_Speech_Recognition_for_Multiple_Languages'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 277 (delta 33), reused 81 (delta 22), pack-reused 177
Receiving objects: 100% (277/277), 69.77 MiB | 29.30 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/Visual_Speech_Recognition_for_Multiple_Languages


In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install scipy
!pip install scikit-image
!pip install av
!pip install six
!pip install mediapipe
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 46.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
from pipelines.model import AVSR
from pipelines.data.data_module import AVSRDataLoader
from pipelines.detectors.mediapipe.detector import LandmarksDetector

class InferencePipeline(torch.nn.Module):
    def __init__(self, modality, model_path, model_conf, detector="mediapipe", face_track=False, device="cuda:0"):
        super(InferencePipeline, self).__init__()
        self.device = device
        # modality configuration
        self.modality = modality
        self.dataloader = AVSRDataLoader(modality, detector=detector)
        self.model = AVSR(modality, model_path, model_conf, rnnlm=None, rnnlm_conf=None, penalty=0.0, ctc_weight=0.1, lm_weight=0.0, beam_size=40, device=device)
        if face_track and self.modality in ["video", "audiovisual"]:
            self.landmarks_detector = LandmarksDetector()
        else:
            self.landmarks_detector = None


    def process_landmarks(self, data_filename, landmarks_filename):
        if self.modality == "audio":
            return None
        if self.modality in ["video", "audiovisual"]:
            landmarks = self.landmarks_detector(data_filename)
            return landmarks


    def forward(self, data_filename, landmarks_filename=None):
        assert os.path.isfile(data_filename), f"data_filename: {data_filename} does not exist."
        landmarks = self.process_landmarks(data_filename, landmarks_filename)
        data = self.dataloader.load_data(data_filename, landmarks)
        transcript = self.model.infer(data)
        return transcript

    def extract_features(self, data_filename, landmarks_filename=None, extract_resnet_feats=False):
        assert os.path.isfile(data_filename), f"data_filename: {data_filename} does not exist."
        landmarks = self.process_landmarks(data_filename, landmarks_filename)
        data = self.dataloader.load_data(data_filename, landmarks)
        with torch.no_grad():
            if isinstance(data, tuple):
                enc_feats = self.model.model.encode(data[0].to(self.device), data[1].to(self.device), extract_resnet_feats)
            else:
                enc_feats = self.model.model.encode(data.to(self.device), extract_resnet_feats)
        return enc_feats

In [ ]:
%mkdir -p /content/data/
!wget http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_V_WER19.1.zip -O /content/data/LRS3_V_WER19.1.zip
!unzip -o /content/data/LRS3_V_WER19.1.zip -d /content/data/

--2024-01-18 07:51:21--  http://www.doc.ic.ac.uk/~pm4115/autoAVSR/LRS3_V_WER19.1.zip
Resolving www.doc.ic.ac.uk (www.doc.ic.ac.uk)... 146.169.13.6
Connecting to www.doc.ic.ac.uk (www.doc.ic.ac.uk)|146.169.13.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 937274463 (894M) [application/zip]
Saving to: ‘/content/data/LRS3_V_WER19.1.zip’

/content/data/LRS3_ 100%[===================>] 893.85M  21.4MB/s    in 44s     

2024-01-18 07:52:06 (20.3 MB/s) - ‘/content/data/LRS3_V_WER19.1.zip’ saved [937274463/937274463]

Archive:  /content/data/LRS3_V_WER19.1.zip
  inflating: /content/data/LRS3_V_WER19.1/model.json  
  inflating: /content/data/LRS3_V_WER19.1/model.pth  


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install moviepy

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-stf1j1iv
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-stf1j1iv
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=faca60a99399b7b295f5355b7a492de09f8c44a1a1d736faadd8602e6e37a2f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-zk93son9/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

**Audio To Text**

In [ ]:
from moviepy.editor import VideoFileClip
import whisper
import os
import logging


class ExtractAudio:
    def __init__(self, video_path, output_audio_path):
        """
        Initializes an instance of the ExtractAudio class.

        Args:
            video_path (str): The path to the video file.
            output_audio_path (str): The path to save the extracted audio file.
        """
        self.video_path = video_path
        self.output_audio_path = output_audio_path
        self.model = whisper.load_model("base")

    def extract_and_transcribe_segment(self, start_time, end_time):
        """
        Extracts audio from a specific segment of the video, transcribes it, and returns the transcribed text.

        Args:
            start_time (float): The start time of the segment in seconds.
            end_time (float): The end time of the segment in seconds.

        Returns:
            str: The transcribed text of the audio segment.
        """
        try:
            with VideoFileClip(self.video_path) as video:
                # Extract only the specified segment
                audio_segment = video.subclip(start_time, end_time)
                audio_segment.audio.write_audiofile(
                    self.output_audio_path, codec="libmp3lame"
                )

                # Transcribe the extracted audio
                result = self.model.transcribe(self.output_audio_path)
                return result["text"]
        except FileNotFoundError:
            logging.error("Audio file not found.")
        except Exception as e:
            logging.error(f"Error in audio extraction and transcription: {e}")
        finally:
            if os.path.exists(self.output_audio_path):
                os.remove(self.output_audio_path)

    def extract_and_transcribe_segments(self, segments):
        """
        Extracts and transcribes audio for multiple segments.

        Args:
            segments (list): A list of tuples representing the start and end times of each segment.

        Returns:
            list: A list of tuples containing the transcribed text, start time, and end time for each segment.
        """
        transcriptions = []
        for start_time, end_time in segments:
            transcription = self.extract_and_transcribe_segment(start_time, end_time)
            transcriptions.append((transcription, start_time, end_time))
        return transcriptions

    def extract_full_audio(self):
        """
        Extracts the full audio from the video.
        """
        try:
            with VideoFileClip(self.video_path) as video:
                video.audio.write_audiofile(self.output_audio_path, codec="libmp3lame")
        except FileNotFoundError:
            print("Video file not found.")

    def transcribe_full_audio(self):
        """
        Transcribes the full audio of the video.

        Returns:
            str: The transcribed text of the full audio.
        """
        try:
            return self.model.transcribe(self.output_audio_path)["text"]
        finally:
            if os.path.exists(self.output_audio_path):
                os.remove(self.output_audio_path)

**Similarity**

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

class BertSimilarity:
    """
    Class for calculating cosine similarity between sentences using BERT embeddings,
    optimized for GPU acceleration.
    """

    def __init__(self):
        # Check if CUDA is available and set the device to GPU or CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load the model and tokenizer
        self.model = BertModel.from_pretrained("bert-base-uncased").to(self.device)
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    def extract_sentences(self, data):
        """
        Extracts sentences from the given data which is a list of tuples.
        Each tuple contains a sentence and its corresponding timestamps.

        :param data: List of tuples, where each tuple is (sentence, start_time, end_time)
        :return: List of sentences.
        """
        return [sentence[0] for sentence in data]

    def calculate_similarity(self, sentences1, sentences2):
        """
        Calculates the cosine similarity between the sentences of two datasets.

        :param sentences1: First set of sentences.
        :param sentences2: Second set of sentences.
        :return: Matrix of cosine similarity scores.
        """
        # Tokenize and prepare the sentences for the BERT model
        inputs1 = self.tokenizer(
            sentences1, padding=True, truncation=True, max_length=512, return_tensors="pt"
        ).to(self.device)

        inputs2 = self.tokenizer(
            sentences2, padding=True, truncation=True, max_length=512, return_tensors="pt"
        ).to(self.device)

        # Process sentences through the model
        with torch.no_grad():
            outputs1 = self.model(**inputs1)
            outputs2 = self.model(**inputs2)

        # Use the [CLS] token embeddings for similarity
        cls_embeddings1 = outputs1[0][:, 0, :]
        cls_embeddings2 = outputs2[0][:, 0, :]

        # Calculate cosine similarity
        similarity = cosine_similarity(
            cls_embeddings1.detach().cpu().numpy(),
            cls_embeddings2.detach().cpu().numpy(),
        )

        return similarity

    def plot_similarity_and_accuracy(self, data1, data2):
        """
        Plots the similarity between two sets of sentences and shows the average accuracy.

        :param data1: The first set of sentences (as a list of tuples).
        :param data2: The second set of sentences (as a list of tuples).
        :return: Cosine similarity score.
        """
        sentences1 = self.extract_sentences(data1)
        sentences2 = self.extract_sentences(data2)

        sentences1 = ' '.join(sentences1)
        sentences2 = ' '.join(sentences2)
        similarities = self.calculate_similarity([sentences1], [sentences2])[0][0]

        return similarities

<div style = "background-color: Red">Lips Detection: Give the ranges in which lips got detected</div>

In [ ]:
!pip uninstall -y opencv-python opencv-python-headless
!pip install opencv-python-headless==4.5.5.64

Found existing installation: opencv-python 4.8.0.76
Uninstalling opencv-python-4.8.0.76:
  Successfully uninstalled opencv-python-4.8.0.76
Found existing installation: opencv-python-headless 4.9.0.80
Uninstalling opencv-python-headless-4.9.0.80:
  Successfully uninstalled opencv-python-headless-4.9.0.80
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 13.6 MB/s eta 0:00:00


In [ ]:
import cv2
import dlib

class LipDetector:

    def __init__(self, video_path, merge_gap_ms=150):
        self.video_path = video_path
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(
            "/content/shape_predictor_68_face_landmarks.dat"
        )
        self.merge_gap_ms = merge_gap_ms

    def process_video(self):
        video = cv2.VideoCapture(self.video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        current_frame = 0
        lips_detected_segments = []
        segment_start = None

        while current_frame < total_frames:
            video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            ret, frame = video.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.detector(gray)
            lips_detected = False

            for face in faces:
                landmarks = self.predictor(gray, face)
                if any(landmarks.part(n) for n in range(48, 68)):
                    lips_detected = True
                    break

            if lips_detected:
                if segment_start is None:
                    segment_start = current_frame / fps
            elif segment_start is not None:
                lips_detected_segments.append((segment_start, current_frame / fps))
                segment_start = None

            current_frame += 1

        video.release()

        # Add the last segment if it ends with lip detection
        if segment_start is not None:
            lips_detected_segments.append((segment_start, total_frames / fps))

        return lips_detected_segments

    def merge_segments(self, lips_detected_segments):
        merged_segments = []
        for start, end in lips_detected_segments:
            if (
                merged_segments
                and start - merged_segments[-1][1] <= self.merge_gap_ms / 1000.0
            ):
                merged_segments[-1] = (merged_segments[-1][0], end)
            else:
                merged_segments.append((start, end))

        return merged_segments

    def detect_lips(self):
        lips_detected_segments = self.process_video()
        return self.merge_segments(lips_detected_segments)


Transcribe

In [ ]:
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip


class VideoTranscriber:

    def __init__(self, video_path, model_conf, model_path, device="cuda:0"):
        self.video_path = video_path
        self.model_conf = model_conf
        self.model_path = model_path
        self.device = device
        self.pipeline = InferencePipeline(
            "video",
            self.model_path,
            self.model_conf,
            face_track=True,
            device=self.device,
        )

    def transcribe(self, lip_detected_segments, chunk_length=3, min_chunk_length=1):

        transcripts = []

        for segment in lip_detected_segments:
            start_time, end_time = segment
            segment_duration = end_time - start_time

            # Check if the segment is shorter than the minimum chunk length
            if segment_duration < min_chunk_length:
                if transcripts:
                    # Extend the previous chunk if there is one
                    transcripts[-1] = (transcripts[-1][0], start_time, end_time)
                continue

            current_time = start_time
            while current_time < end_time:
                chunk_end_time = min(current_time + chunk_length, end_time)
                chunk_filename = f"chunk_{current_time}_{chunk_end_time}.mp4"
                ffmpeg_extract_subclip(
                    self.video_path,
                    current_time,
                    chunk_end_time,
                    targetname=chunk_filename,
                )

                try:
                    transcript = self.pipeline(chunk_filename)
                    transcripts.append((transcript, current_time, chunk_end_time))
                except Exception as e:
                    print(f"Error processing chunk {chunk_filename}: {e}")
                finally:
                    if os.path.exists(chunk_filename):
                        os.remove(chunk_filename)

                current_time += chunk_length

        return transcripts


In [ ]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2FO2IsRIOoPTMJdkAofEdRoZtnb_7DJJ7H9MfWmG9QP8hx8SW

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=79004f09948816bd2e9a4faff6b6b832f70bdde57c96fe307a79431af4f15a8c
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)
run_with_ngrok(app)

UPLOAD_FOLDER = '/content/uploaded_videos'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

model_conf = "/content/data/LRS3_V_WER19.1/model.json"
model_path = "/content/data/LRS3_V_WER19.1/model.pth"

# Ensure the upload folder exists
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

global video_path
video_path = None

@app.route('/lipDetection', methods=['GET'])
def lipsdetector():
    '''
    Detects lips in the video and returns the merged lip detection segments.
    '''

    global video_path  # Use the global keyword to access the global variable

    lip_detector = LipDetector(video_path)
    lips_detected_segments = lip_detector.detect_lips()

    # Returning the data with a specific key
    return jsonify({"lips_detected_segments": lips_detected_segments})

@app.route('/videoTranscribe', methods=['GET', 'POST'])
def videoTranscribe():
    """
    Transcribes video segments based on lips_detected_segments received from the request.
    """
    global video_path  # Use the global keyword to access the global variable
    data = request.get_json()
    lips_detected_segments = data.get('lips_detected_segments')
    if not lips_detected_segments:
        return jsonify({"error": "No lips_detected_segments provided"}), 400

    # Assuming you pass the lips_detected_segments directly to the VideoTranscriber
    transcriber = VideoTranscriber(video_path, model_conf, model_path)
    transcripted_text = transcriber.transcribe(
        lips_detected_segments, chunk_length=3, min_chunk_length=1
    )
    return jsonify({"transcripted_text": transcripted_text})

@app.route('/audioTranscribe', methods=['GET', 'POST'])
def audioTranscribe():
    """
    Extracts and transcribes audio from video segments.
    """
    global video_path  # Use the global keyword to access the global variable
    # video_path = request.args.get('video_path')
    data = request.get_json()
    lips_detected_segments = data.get('lips_detected_segments')
    if not lips_detected_segments:
        return jsonify({"error": "No lips_detected_segments pro"})

    audio_transcriber = ExtractAudio(video_path, "temp_audio.wav")
    extracted_text = audio_transcriber.extract_and_transcribe_segments(
        lips_detected_segments
    )
    return jsonify({"extracted_text": extracted_text})

@app.route('/bertSimilarity', methods=['GET', 'POST'])
def similarity():
    data = request.get_json()

    transcripted_text = data.get('transcripted_text')
    extracted_text = data.get('extracted_text')

    bert_similarity = BertSimilarity()

    similarity_scores = bert_similarity.plot_similarity_and_accuracy(
        transcripted_text, extracted_text
    )
    similarity_score = str(similarity_scores)
    return jsonify({"score": similarity_score})

@app.route('/upload', methods=['GET', 'POST'])
def video_upload():

    global video_path  # Reference the global variable

    if 'video' not in request.files:
        return jsonify({"error": "No video file provided"})

    uploaded_file = request.files['video']

    if uploaded_file.filename == '':
        return jsonify({"error": "No selected file"})

    if uploaded_file:
        filename = secure_filename(uploaded_file.filename)
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        uploaded_file.save(video_path)
        return jsonify({"message": "Video uploaded successfully", "path": video_path})

    return jsonify({"error": "File upload failed"})

@app.route('/delete', methods=['GET', 'POST'])
def deleteVideo():
    # video_path = request.args.get('video_path') # or request.form['video_path']
    if os.path.exists(video_path):
        os.remove(video_path)
        return jsonify({"message": "Video deleted successfully"})
    else:
        return jsonify({"error": "Video not found"})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d57c-34-125-190-8.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
